## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-26-03-45-45 +0000,nyt,"James Comey, Former FBI Director, Indicted Aft...",https://www.nytimes.com/2025/09/25/us/politics...
1,2025-09-26-03-45-00 +0000,wsj,Tokyo’s Price Growth Steady But Rate Increase ...,https://www.wsj.com/economy/central-banking/to...
2,2025-09-26-03-26-01 +0000,bbc,Comey indictment escalates Trump's promise of ...,https://www.bbc.com/news/articles/c33r872egvjo...
3,2025-09-26-03-20-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...
4,2025-09-26-02-59-16 +0000,nypost,Grand Central Terminal power outage rattles co...,https://nypost.com/2025/09/25/us-news/grand-ce...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2291/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,53
20,president,18
197,israel,13
321,government,12
80,ice,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
123,2025-09-25-21-02-00 +0000,wsj,Fed governor Lisa Cook urged the Supreme Court...,https://www.wsj.com/politics/policy/fed-govern...,138
37,2025-09-26-01-00-00 +0000,wsj,The Supreme Court is poised to decide whether ...,https://www.wsj.com/economy/central-banking/fe...,136
3,2025-09-26-03-20-00 +0000,wsj,President Trump announced many new tariffs Thu...,https://www.wsj.com/economy/trade/trump-to-sla...,110
77,2025-09-25-22-50-53 +0000,nypost,Trump vows he ‘will not allow Israel to annex ...,https://nypost.com/2025/09/25/world-news/trump...,109
201,2025-09-25-16-25-34 +0000,nypost,Disney gears up for court battle with Trump af...,https://nypost.com/2025/09/25/business/disney-...,108


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
123,138,2025-09-25-21-02-00 +0000,wsj,Fed governor Lisa Cook urged the Supreme Court...,https://www.wsj.com/politics/policy/fed-govern...
9,77,2025-09-26-02-31-54 +0000,nypost,Anti-ICE shooter was lazy stoner who relocated...,https://nypost.com/2025/09/25/us-news/anti-ice...
217,64,2025-09-25-15-21-00 +0000,wsj,The clock is ticking down to an Oct. 1 deadlin...,https://www.wsj.com/politics/policy/government...
75,57,2025-09-25-22-53-19 +0000,nypost,Former FBI Director James Comey indicted on ch...,https://nypost.com/2025/09/25/us-news/former-f...
193,55,2025-09-25-16-58-18 +0000,cbc,Liberal government instructs Canada Post to en...,https://www.cbc.ca/news/politics/canada-post-t...
203,49,2025-09-25-16-17-15 +0000,nypost,Over 100 ex-ABC journalists urge Disney’s Bob ...,https://nypost.com/2025/09/25/business/over-10...
284,45,2025-09-25-10-49-00 +0000,wsj,Denmark Says New Drone Flights Over Military B...,https://www.wsj.com/world/europe/denmark-says-...
135,38,2025-09-25-20-16-00 +0000,wsj,U.S. companies are facing heightened scrutiny ...,https://www.wsj.com/politics/policy/h1b-visa-g...
60,37,2025-09-25-23-48-01 +0000,nypost,"Anti-ICE gunman Joshua Jahn spent 17,500 hours...",https://nypost.com/2025/09/25/us-news/joshua-j...
275,34,2025-09-25-11-24-18 +0000,bbc,French navy seizes nearly 10 tonnes of cocaine...,https://www.bbc.com/news/articles/c15k7j9ken4o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
